In [ ]:
pip install --upgrade "langchain>=0.1.0" "langchain-community>=0.0.29" "langchain-openai>=0.0.5" "openai>=1.12.0" "chromadb>=0.4.22" "langchain-chroma>=0.0.1" "python-dotenv>=1.0.0"

In [1]:
# 벡터 임베딩
# Chroma 사용

import csv
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from dotenv import load_dotenv
import os

# .env 파일 로드
load_dotenv()

open_ai_api_key = os.getenv('OPENAI_API_KEY')
course_csv_path = os.getenv('COURSE_CSV_PATH')

embedding_model = OpenAIEmbeddings(
    model = "text-embedding-3-large",
    openai_api_key = open_ai_api_key,
    request_timeout = 3000,
)

raw_documents = []
with open(course_csv_path,'r', encoding='utf-8-sig') as fh:
    reader = csv.DictReader(fh)
    for row in reader:
        course_text = f"{row['Course_Name']}: {row['Course_Description']}, {row['Course_Teacher']}"
        document = Document(
            page_content= course_text,
            metadata= {
                'course_id': row['Course_Id']
            },
            id=row['Course_Id']
        )
        raw_documents.append(document)

vector = Chroma.from_documents(
    documents=raw_documents,
    embedding=embedding_model, 
    persist_directory='./course_vector_db', 
    collection_name="courses"
)

In [2]:
# Chroma DB에 접근하는 코드
from langchain_chroma import Chroma
from chromadb import Client, Settings

# 클라이언트 생성
client = Client(Settings(
    persist_directory='./course_vector_db',
    is_persistent=True
))

# 컬렉션 가져오기
collection = client.get_collection(name="courses")

# 컬렉션의 모든 항목 가져오기
results = collection.get(include=['embeddings', 'documents', 'metadatas'])

# 결과 확인
if results:
    print(f"저장된 문서 수: {len(results['ids'])}")
    # 처음 몇 개의 문서 출력
    for i in range(min(3, len(results['ids']))):
        print(f"\n문서 ID: {results['ids'][i]}")
        print(f"문서 내용: {results['documents'][i]}")
        print(f"메타데이터: {results['metadatas'][i]}")
        print(f"임베딩: {results['embeddings'][i].tolist()}")
else:
    print("컬렉션이 비어있거나 접근할 수 없습니다.")

저장된 문서 수: 212

문서 ID: 10000
문서 내용: C#과 유니티로 배우는 게임 개발 올인원 패키지 Online.: "1. 수업 목표 :\n- 게임 개발에 대한 기초 지식을 습득하고, 배운 지식을 바탕으로 문제를 해결할 수 있는 사고능력을 기릅니다.\n- 게임 개발자 취업에 필요한 모든 과정을 하나의 강의로 정복합니다.\n- 프로그래밍 기초는 탄탄히, 게임 엔진 실무를 기반으로 하는 포트폴리오까지 준비할 수 있습니다.\n\n2. 강의요약 :\n- 게임 회사 코딩테스트 대비부터 포트폴리오까지 한 번에 수강할 수 있습니다.\n- 현직 게임 개발자가 게임 개발자 취업의 A to Z를 직접 강의합니다.\n\n3. 특이사항 : 없음", "한창민 강사\n[주요 경력사항]\n- 現 M사 – 프로그래밍 팀장 겸 PM\n- 前 위메이드 – MMORPG 게임 선임 연구원\n- 前 바른손 – 모바일 게임 개발 (보석팡, 아쿠아빌리지 for Kakao)\n- 前 넷이즈(중국) – SNS 게임 프로그래밍 팀장\n\n김민백 강사\n[주요 경력사항]\n- 現 ’긱블’메이커 / PD\n- 現 ‘민바크의 게임 제작’ 유튜브 채널 운영 중\n\n배병도 강사\n[주요 경력사항]\n- 前 스마일게이트스토브 - 차장(C# / C++ 라이브러리 제작)\n- 前 4:33 (Lost Kingdom 해외 퍼블리싱 지원, 활2 최적화 작업 지원)\n- 前 iBH Soft - 과장(Running M 게임 개발)\n- 前 스노우 파이프 - 부장(사쿠라대전 클라이언트 개발 총괄)\n- 前 ANB Soft - 과장(애니머스 온라인 클라이언트 개발 총괄, 드래곤 키우기 클라이언트 개발 총괄)\n- 前 CJIG(넷마블계열사) - 대리\n\n손정현 강사\n[주요 경력사항]\n- 現 인바디 PC어플리케이션 개발(C# 개발)"
메타데이터: {'course_id': '10000'}
임베딩: [-0.003050442785024643, 0.01047248113900423, -0.019844505935907364, 6

In [18]:
# 또는 LangChain의 Chroma를 사용할 경우:
db = Chroma(
    persist_directory='./course_vector_db',
    embedding_function=embedding_model,
    collection_name="courses"
)



# 간단한 검색 테스트
print(f"시맨틱 검색")
search_query = "Recommend me a data analysis course"
print(f"검색 쿼리: {search_query}")
results = db.similarity_search_with_relevance_scores(search_query, k=5)
for doc, score in results:
    print(f"유사도 점수: {score:.4f}")  # 소수점 4자리까지 표시
    print(f"강좌명: {doc.page_content}")
    print(f"강좌 ID: {doc.metadata['course_id']}")
    print("-" * 50)

시맨틱 검색
검색 쿼리: Recommend me a data analysis course
유사도 점수: 0.2785
강좌명: 데이터 분석을 위한 기초 통계 완전 정복: "1. 수업 목표\n- 데이터 분석을 시작하기 위해 필요한 수학, 통계 기초를 알려 드립니다. \n- 통계에 대한 두려움을 없앱니다. \n- 실습 예제를 통해 이론 지식을 쉽게 습득할 수 있습니다. \n\n2. 강의 요약\n- 데이터 분석을 시작하기에 앞서, 통계 기초부터 데이터 탐색(EDA)까지 엑셀 실습을 통해 배울 수 있는 강의", "강사님 약력\n현) FINDA(핀다) 데이터 사이언티스트\n전) 특허 데이터 분석 기업 선임연구원\n- 연세대학교 수학과 석박사 통합과정\n- 연세대학교 수학과 학사 졸업"
강좌 ID: 10043
--------------------------------------------------
유사도 점수: 0.2403
강좌명: 나의 커리어 치트키 데이터 분석 유치원: "1. 수업 목표 - 왕초보들에게 필요한 파이썬, 통계기초 부터 시작해서 실무에 필요한 데이터 분석 이론, 실무 프로젝트까지 모두 마스터해서 바로 실무에 사용가능하도록 구성했습니다.\n\n2. 강의 요약 - 1) 데이터와 친해지기 > 2) 파이썬 익숙해지기 > 3) 데이터 분석 능숙해 지기 > 4) 프로젝트 실습으로 배우는 실무 데이터 분석 마스터", "권정민 님\n이력\n전) ODK Media, Inc / Senior Data Scientist, Data Analytics Lead\n전) 우아한 형제들 / Data Scientist\n전) Viva Repulica / Data Scientist\n전) SK planet / Data Analyst\n전) KT NexR / Data Scientist\n전) Samsung Electronics / Data Analyst\n전) Nexon / Data Analyst\n전) Oracle / Business Analyst\n\n출판\n- ‘데이터 분석가의 숫자유